# 4.B. Recommender -- User

We'll test out our album review recommendation engine in this notebook before incorporating into a hybrid approach in notebook **4D**.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import requests
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import lyricsgenius
import re
import sys
import spotipy
import spotipy.util as util
from sklearn.compose import ColumnTransformer, make_column_transformer
from pyjarowinkler import distance
import nltk

import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import requests
import seaborn as sns
import warnings
import matplotlib.pyplot as plt

import nltk
from nltk.stem import PorterStemmer
from sklearn.model_selection import cross_val_score

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from gensim import corpora, models



In [2]:
users = pd.read_csv('follower_genius_500k.csv')

In [3]:
users['artist_clean'] = users['artist_name'].str.replace('$', 'S')
users['artist_clean'] = users['artist_clean'].str.replace("’", "")
users['artist_clean'] = users['artist_clean'].str.replace('&', 'and')
users['artist_clean'] = users['artist_clean'].str.replace('!','') 
users['artist_clean'] = users['artist_clean'].str.replace('([ ]{2,})', ' ', regex = True) 
users['artist_clean'] = users['artist_clean'].str.strip()
users['artist_clean'] = users['artist_clean'].str.lower()

In [4]:
users = users.drop(columns=['follower_role'])

In [5]:
users = users.dropna()

In [6]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances### Genius -Follower DF Creation

In [7]:
user_pivot = users.pivot_table(index='artist_clean', columns = ['follower_name'], values = 'follow')

In [8]:
user_pivot.head()

follower_name,$hamyr,$haz,$mokeyyy,$wag Hip-Hop,",KKKKKKKLLLKK",.z3r0.,0-0-J_D_O_N-0-0,0-MXXL-0,0-Schwa-0,0-efray-0,...,‌bluii,‍ lyra messier,⁠kartashov,☀️ ✼ ☆ KayneTheDog ☆ ✼ ☀️,☣̭̰♜͙̟͇͓́ɤ͏̪̦̬̻͚̗̲Վ̘̰̥̫̜͔a̻͕͞ņ̠̟̩̼̰̲̰͞ ̖̘͇͍͎ͅ'̠̠̻̹M͚̠̣̺̬͕͞.̡̫̗O͎̠̺̪̻̺͚͟.̖̖͍̟͍̜͔M̘̖͙̝͠.̯͙͎͈̗̯͜T̺.̟̮̫̜̘͞'͕̜͢ ̛̳̫̗̬͈V͚̰̬̦༏̢̝̭̰ͅc̴̩̰̺̤̞͇͖e̹̫̮̳̥̣̘♜͙̟͇͓́☣̭̰,✞☭ Perc Nowitzki ☭✞,✰MAGZEN✰,🌴TruSwag🌴,💎 YBN Mystique 💎,🚀👑ASTROWORLD
artist_clean,,,,,,,,,,,,,,,,,,,,,
(hed) p.e.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
03 greedo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
070 shake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2 chainz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [9]:
user_pivot_sparse = sparse.csr_matrix(user_pivot.fillna(0))

In [10]:
recommender = pairwise_distances(user_pivot_sparse, metric='cosine')
recommender_df = pd.DataFrame(recommender, columns=user_pivot.index, index=user_pivot.index)
recommender_df.shape

(1166, 1166)

In [11]:
recommender_df

artist_clean,(hed) p.e.,03 greedo,070 shake,100s,2 chainz,2 live crew,2 pistols,21 savage,2mex,2pac,...,​billy woods,​cupcakke,​ilovemakonnen,​ilovememphis,​j-hope,​milo,"​nothing,nowhere.",​tobymac,​​alt-j,​​wifisfuneral
artist_clean,,,,,,,,,,,,,,,,,,,,,
(hed) p.e.,0.000000,1.000000,1.000000,1.000000,0.995301,1.000000,1.000000,1.000000,1.000000,0.989393,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
03 greedo,1.000000,0.000000,0.983999,1.000000,1.000000,1.000000,1.000000,0.987517,1.000000,0.993682,...,1.000000,1.000000,0.988284,0.965435,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
070 shake,1.000000,0.983999,0.000000,1.000000,0.988894,1.000000,1.000000,0.979360,1.000000,0.990598,...,1.000000,0.986129,0.982566,0.982855,1.000000,0.995154,0.995066,1.000000,0.989444,1.000000
100s,1.000000,1.000000,1.000000,0.000000,0.956194,1.000000,1.000000,0.996382,1.000000,0.986265,...,0.971950,1.000000,0.959249,0.969944,1.000000,0.991505,1.000000,1.000000,0.993832,1.000000
2 chainz,0.995301,1.000000,0.988894,0.956194,0.000000,0.991953,0.972124,0.933569,0.994491,0.923524,...,1.000000,0.992721,0.951206,0.988004,1.000000,0.989829,0.998274,0.998465,0.992614,0.982099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
​milo,1.000000,1.000000,0.995154,0.991505,0.989829,1.000000,1.000000,0.994959,0.980769,0.984692,...,0.960923,1.000000,0.992904,1.000000,1.000000,0.000000,1.000000,1.000000,0.991406,0.988973
"​nothing,nowhere.",1.000000,1.000000,0.995066,1.000000,0.998274,1.000000,1.000000,0.991018,1.000000,1.000000,...,1.000000,0.991377,0.989161,1.000000,0.994664,1.000000,0.000000,1.000000,0.993437,0.988772
​tobymac,1.000000,1.000000,1.000000,1.000000,0.998465,1.000000,1.000000,0.997717,1.000000,0.996534,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000


In [12]:
users.head()


,artist_api_path,follower_name,follower_id,follower_api_path,follow,artist_name,artist_url,artist_clean
0,/artists/130,Origiginal,4305077.0,/users/4305077,1,Drake,https://genius.com/artists/Drake,drake
1,/artists/130,JWTM,75379.0,/users/75379,1,Drake,https://genius.com/artists/Drake,drake
2,/artists/130,Jake Learned,13931.0,/users/13931,1,Drake,https://genius.com/artists/Drake,drake
3,/artists/130,​blustery,4469304.0,/users/4469304,1,Drake,https://genius.com/artists/Drake,drake
4,/artists/130,ASAP_Boli,7840721.0,/users/7840721,1,Drake,https://genius.com/artists/Drake,drake


In [21]:
search = 'kendrick lamar'

recommender_df[search].sort_values(ascending=True)[1:11]


artist_clean
schoolboy q          0.629146
asap rocky           0.633423
nas                  0.656188
j. cole              0.667594
chance the rapper    0.671670
big sean             0.690982
ab-soul              0.697010
childish gambino     0.700112
kanye west           0.707621
pusha t              0.722092
Name: kendrick lamar, dtype: float64